In [1]:
from scipy.sparse.linalg import eigs
import scipy.sparse as sparse
import numpy as np
import numpy.linalg as la
import sympy as sp

import matplotlib.pyplot as plt
%matplotlib inline

import Potapov_Code.Time_Delay_Network as networks
from Potapov_Code.functions import spatial_modes
from decimal import Decimal

from sympy import init_printing
init_printing() 

from sympy.utilities.autowrap import ufuncify
import time

In [2]:
import Potapov_Code.Time_Delay_Network

In [3]:
from Potapov_Code.functions import gcd_lst

In [4]:
X1 = Potapov_Code.Time_Delay_Network.Example3()

In [6]:
def _find_commensurate(delays):
    '''
    Find the 'gcd' but for Decimal numbers.

    Args:
        delays(list of Demicals): numbers whose gcd will be found.

    Returns:
        Decimal gcd.
    '''
    mult = min([d.as_tuple().exponent for d in delays])
    power = 10**-mult
    delays = map(lambda x: x*power,delays)
    int_gcd = gcd_lst(delays)
    return int_gcd/power

Decimal_delays = map(lambda x: Decimal(str(x)),X1.delays)
Decimal_gcd = _find_commensurate(Decimal_delays)

xs = [sp.symbols('x_'+str(i)) for i in range(4) ]
E_sym = sp.Matrix(np.zeros_like(X1.M1))
for i,delay in enumerate(Decimal_delays):
    E_sym[i,i] = xs[i]
M1_sym = sp.Matrix(X1.M1)
num, den = (E_sym - M1_sym).det().as_numer_denom()

### Alternatively, we can Taylor expand about the $z$ variable only. This gives just Newton's method. To first order we get
$$
\Delta z = - \frac{f(z,T+\Delta T)}{\frac{\partial f}{\partial z (z,T+\Delta T}}.
$$
This can be used as an iterative procedure with the substitution $z \mapsto z + \Delta_z$. In addition, dependence on $\Delta T $ can be incorporated by evaluation $\Delta T \equiv \Delta T(z + \Delta z)$.

In [7]:
num, den

In [54]:
z, z_Delta = sp.symbols('z dz',complex=True)
Ts = [sp.symbols('T_'+str(i),real=True) for i in range(4)]
x,y = sp.symbols('x y', real = True)

In [16]:
num2 = num.subs({x: sp.exp(-z*T) for x,T in zip(xs,Ts)})

In [48]:
num2

In [17]:
num3 = num2.subs({sp.exp(-z*T): sp.cos(-x*T)*(1j*sp.sin(y*T)+sp.cos(y*T)) for T in Ts})

In [49]:
num3

In [18]:
num_real,num_imag = num3.expand().as_real_imag()

In [44]:
[x,y]+[Ts]

In [122]:
f_r =  ufuncify( [x,y]+Ts, num_real)
f_i =  ufuncify( [x,y]+Ts, num_imag)

In [47]:
f_r(1,2,3,4,5,6)+f_i(1,2,3,4,5,6)*1j

(0.14730830564958425-0.40346430265728966j)

In [52]:
## testing
num3.subs({x:1,y:2,Ts[0]:3,Ts[1]:4,Ts[2]:5,Ts[3]:6}).evalf()

In [89]:
#Consolidating
def get_real_imag_func(expression):
    '''
    Takes an expression in terms of Ts and sp.exp(-z*T) for T in Ts. 
    Here :math:`z = x + i y` is a complex number.
    '''
    D = {sp.exp(-z*T): sp.cos(-x*T)*(1j*sp.sin(y*T)+sp.cos(y*T)) for T in Ts}
    expression2 = expression.subs(D)
    num_real,num_imag = expression2.expand().as_real_imag()
    f_r =  ufuncify( [x,y]+Ts, num_real)
    f_i =  ufuncify( [x,y]+Ts, num_imag)
    return lambda x,y,Ts: f_r(x,y,*Ts)+f_i(x,y,*Ts)*1j

In [90]:
F = get_real_imag_func(num2)

In [91]:
##  testing
F(1,2,(3,4,5,6))

(0.14730830564958425-0.40346430265728966j)

In [104]:
get_real_imag_func(sp.diff(num2,z))(1,2,(3,4,5,6))

(-0.33226636779973417+1.7256217430980794j)

In [117]:
def blah(args):
    for i in range(500):
        F(*args)

In [161]:
ls = [[0.]*5]*5

In [163]:
ls[1][2] = 4

In [164]:
ls

In [165]:
ls2 = np.zeros((5,5))

In [166]:
ls2

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [169]:
ls2[2,3] = 4

In [181]:
ls2

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [174]:
ls2[2]

array([ 0.,  0.,  0.,  4.,  0.])

In [175]:
delays = [1,4,5,6,7]

In [179]:
map(sum,zip(ls2[2],delays))

In [183]:
round(1.111111,2)